In [1]:
from poloniex import Poloniex

polo = Poloniex()

In [2]:
markets = polo.returnTicker()
pairs = []
for name, thinks in markets.items():
    pairs.append(name)

In [6]:
# funciones necesarias para el manejo del tiempo
from datetime import datetime, timedelta
from time import sleep
from tusp import string2ts
from bot import prepareData
import numpy as np
import pylab as pl
%matplotlib inline

class PairData:
    
    def __init__(self, pair, period, weeks = 4.3*3, backto = None):
            
        self.pair = pair
        self.period = period
        delta = timedelta(weeks = weeks)
        self.delta = delta
        start = ""
        end = ""

        # Sí el tiempo final es diferente al actual...
        if backto != None:
            to = datetime.strptime(backto, "%Y-%m-%d %H:%M:%S")
            self.to = to
            start = string2ts(to.strftime('%Y-%m-%d %H:%M:%S'))
            
            tf = to + delta
            self.tf = tf
            end = string2ts(tf.strftime('%Y-%m-%d %H:%M:%S'))
            
        else:
            tf = datetime.now()
            self.tf = tf
            end = string2ts(tf.strftime('%Y-%m-%d %H:%M:%S'))
            
            to = tf-delta
            self.to = to
            start = string2ts(to.strftime('%Y-%m-%d %H:%M:%S'))
                              

        self.start = start
        self.end = end
        df = prepareData(pair=pair, start=start, end=end, period=int(period))
        
        self.df = df
        self.market_return()
        self.cum_volume()
    
    def market_return(self):
        """Calcula el retorno del mercado vía log_retornos
        """
        
        serie = self.df["close"]

        # calculando los log retornos
        log_return = np.log(serie).diff().fillna(0)

        # calculando log retorno acumulado
        cum_logr = log_return.cumsum()
        cum_r = np.exp(cum_logr) - 1
        
        self.l_return = log_return
        self.m_return = cum_r
        
    def cum_volume(self):
        """Calcula el volumen acumulado del mercado
        """
        serie = self.df["volume"]
        self.c_volume = serie.cumsum()[-1]
        
    def sortino(self, log = False):
        """ Calcula en ratio de Sortino
        
        """
        
        if log:
            rets = self.l_return
         
        else:
            rets = self.m_return

        semi_var = rets[rets < 0] ** 2
        semi_var = semi_var.sum()/len(rets)
        sortino = np.sqrt(semi_var)

        return rets.mean()/sortino

    def sharpe(self, period_ret = 0.2):
        """
        Calculate the annualised Sharpe ratio of a returns stream 
        based on a number of trading periods, N. N defaults to 252,
        which then assumes a stream of daily returns.

        The function assumes that the returns are the excess of 
        those compared to a benchmark.
        """
        N = self.delta.days
        returns = self.l_return
        excess_ret = returns - period_ret/N
        return np.sqrt(N) * excess_ret.mean() / excess_ret.std()
    
dgb = PairData("BTC_DGB", 3600*4)

print dgb.c_volume

69376.2018248


In [14]:
# cálculo del ratio de Sortino del último mes
volumes = []
pairs = [x for x in pairs if x.split("_")[0] == "BTC"]
for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4, 4.28)
    volume = asset.c_volume
    volumes.append([pair, volume])
    #print sortino

volumes.sort(key = lambda l:l[1], reverse=True)

In [16]:
print volumes[:6]

[[u'BTC_XRP', 495529.69412624009], [u'BTC_ETH', 271784.43474094005], [u'BTC_STR', 156668.32426760995], [u'BTC_BCH', 130337.04435231998], [u'BTC_LTC', 74643.355729889969], [u'BTC_XEM', 59523.939117570015]]


In [44]:
# cálculo del ratio de Sortino del último mes
sortinos = []
pairs = [x for x in pairs if x.split("_")[0] == "BTC"]
for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4, 4.28)
    sortino = asset.sortino(log = True)
    sortinos.append([pair, sortino])
    #print sortino

sortinos.sort(key = lambda l:l[1], reverse=True)

# Cálculo del ratio de Sortino del penúltimo mes
sortinos2 = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4, 4.28, "2017-11-14 00:00:00")
    sortino = asset.sortino(log = True)
    sortinos2.append([pair, sortino])
    #print sortino

sortinos2.sort(key = lambda l:l[1], reverse=True)

# Cálculo del ratio de Sortino del penúltimo mes
sortinos3 = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4, 4.28, "2017-10-14 00:00:00")
    sortino = asset.sortino(log = True)
    sortinos3.append([pair, sortino])
    #print sortino

sortinos3.sort(key = lambda l:l[1], reverse=True)

In [50]:
print "Octubre a Noviembre"
print sortinos3[:6]
print "Noviembre a Diciembre"
print sortinos2[:6]
print "Diciembre a Enero"
print sortinos[:6]

Octubre a Noviembre
[[u'BTC_BCH', 0.22451410688015308], [u'BTC_VTC', 0.1870550227321274], [u'BTC_DASH', 0.093656264314634044], [u'BTC_XBC', 0.093219531216916479], [u'BTC_POT', 0.086388410427155365], [u'BTC_XCP', 0.078599806651906523]]
Noviembre a Diciembre
[[u'BTC_NXT', 0.26422621625796877], [u'BTC_EMC2', 0.23854823526497479], [u'BTC_LTC', 0.16497360354142646], [u'BTC_STR', 0.12273930740126378], [u'BTC_ARDR', 0.10830570372657844], [u'BTC_SBD', 0.093234841773830082]]
Diciembre a Enero
[[u'BTC_ETH', 0.2968386047989543], [u'BTC_ZRX', 0.26198780560861623], [u'BTC_SC', 0.21605898190507009], [u'BTC_XRP', 0.1849220905214137], [u'BTC_STR', 0.18220982102811212], [u'BTC_PASC', 0.17484072513955945]]


In [32]:
# cálculo del ratio de Sharpe del último mes
sharpes = []
for pair in pairs:
    asset_past = PairData(pair, 3600*4, 4.28, "2017-11-14 00:00:00")
    ret = asset_past.m_return[-1]
    asset = PairData(pair, 3600*4, 4.28)
    sharpe = asset.sharpe(ret)
    sharpes.append([pair, sharpe])

sharpes.sort(key = lambda l:l[1], reverse=True)

# Cálculo del ratio de Sharpe del penúltimo mes
sharpes2 = []

for pair in pairs:
    asset_past = PairData(pair, 3600*4, 4.28, "2017-10-14 00:00:00")
    ret = asset_past.m_return[-1]
    asset = PairData(pair, 3600*4, 4.28, "2017-11-14 00:00:00")
    sharpe = asset.sharpe()
    sharpes2.append([pair, sharpe])

sharpes2.sort(key = lambda l:l[1], reverse=True)

# Cálculo del ratio de Sharpe del penúltimo mes
sharpes3 = []

for pair in pairs:
    asset_past = PairData(pair, 3600*4, 4.28, "2017-09-14 00:00:00")
    ret = asset_past.m_return[-1]
    asset = PairData(pair, 3600*4, 4.28, "2017-10-14 00:00:00")
    sharpe = asset.sharpe()
    sharpes3.append([pair, sharpe])

sharpes3.sort(key = lambda l:l[1], reverse=True)

C:\Users\Daniel.LAPTOP-OBVONK0N\Anaconda2\lib\site-packages\ipykernel_launcher.py:55: RuntimeWarning: divide by zero encountered in log


In [51]:
print "Octubre a Noviembre"
print sharpes3[:6]
print "Noviembre a Diciembre"
print sharpes2[:6]
print "Diciembre a Enero"
print sharpes[:6]

Octubre a Noviembre
[[u'BTC_VTC', 0.022409234745069333], [u'BTC_BCH', -0.0099032533503816637], [u'BTC_XBC', -0.3079693882455472], [u'BTC_STR', -0.35762187643243576], [u'BTC_POT', -0.36027701757483549], [u'BTC_XVC', -0.41861861148476942]]
Noviembre a Diciembre
[[u'BTC_EMC2', 0.31511756276026581], [u'BTC_NXT', 0.12549707050650685], [u'BTC_SBD', -0.1440042967538569], [u'BTC_BTM', -0.28932475436845062], [u'BTC_RIC', -0.32467172845924591], [u'BTC_STR', -0.35449899340094249]]
Diciembre a Enero
[[u'BTC_XBC', 2.828188941482245], [u'BTC_BTCD', 2.5938002021163626], [u'BTC_BCH', 2.3499693654578735], [u'BTC_GAS', 2.295426558259563], [u'BTC_GAME', 2.2057114007360785], [u'BTC_ZEC', 2.1989075064991264]]


In [19]:
sortinos3 = [x for x in sortinos if x[0].split("_")[0] == "BTC"]
print sortinos3[0:7]

[[u'BTC_ETH', 0.30196883204988501], [u'BTC_ZRX', 0.26532056873400023], [u'BTC_SC', 0.2214773012516347], [u'BTC_XRP', 0.19425596317204893], [u'BTC_STR', 0.18607695235720123], [u'BTC_STEEM', 0.181236878630986], [u'BTC_BCY', 0.17820544291233495]]


In [20]:
pairs = [x for x in pairs if x.split("_")[0] == "BTC"]

In [21]:
from indicators import rsi
rsi_list = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*24, weeks = 4.3*6)
    RSI1, RSI2 = rsi(asset.df["close"])
    rsi_list.append([pair, RSI1[-1]])
    #print sortino

rsi_list.sort(key = lambda l:l[1], reverse=False)

print rsi_list[0:6]

[[u'BTC_VTC', 37.974411292336271], [u'BTC_NXT', 38.866925910832393], [u'BTC_SBD', 41.359837682122276], [u'BTC_BTCD', 41.664773024097094], [u'BTC_BTS', 41.796096416295697], [u'BTC_POT', 42.259544183424119]]
